In [11]:
pip install requests beautifulsoup4 pandas



SyntaxError: invalid syntax (3768125956.py, line 1)

In [46]:
pip install webdriver-manager


Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install selenium


In [54]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

def scrape_yelp(url):
    # Set up Selenium WebDriver
    options = Options()
    options.add_argument("--headless")  # Run in headless mode
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    # Open the Yelp page
    driver.get(url)
    time.sleep(5)  # Allow JavaScript to load

    # Extract Restaurant Name
    try:
        restaurant_name = driver.find_element(By.CLASS_NAME, "y-css-olzveb").text
        print(f"Restaurant Name: {restaurant_name}")
    except Exception as e:
        print("❌ Failed to get Restaurant Name", e)
        restaurant_name = "Not Found"

    # Extract Total Reviews Count
    try:
        reviews_count = driver.find_element(By.CLASS_NAME, "y-css-yrt0i5").text
        print(f"Total Reviews Count: {reviews_count}")
    except Exception as e:
        print("❌ Failed to get Total Reviews Count", e)
        reviews_count = "Not Found"

    # Extract Reviews
    reviews = []
    review_blocks = driver.find_elements(By.XPATH, "//li[contains(@class,'margin-b')]")

    if not review_blocks:
        print("❌ No reviews found! Check if the page loads reviews dynamically.")

    for block in review_blocks:
        try:
            review_text = block.find_element(By.XPATH, ".//span[contains(@class,'raw__09f24__T4Ezm')]").text
        except:
            review_text = "No Review Text"

        try:
            reviewer = block.find_element(By.XPATH, ".//a[contains(@class,'y-css-1x1e1r2')]").text
        except:
            reviewer = "Anonymous"

        try:
            rating_element = block.find_element(By.XPATH, ".//div[contains(@aria-label,'star rating')]")
            rating = rating_element.get_attribute("aria-label")
        except:
            rating = "No Rating"

        print(f"📌 Review: {review_text[:50]}... | Reviewer: {reviewer} | Rating: {rating}")
        reviews.append([restaurant_name, reviews_count, reviewer, review_text, rating])

    # Close the browser
    driver.quit()

    # Save to CSV
    df = pd.DataFrame(reviews, columns=["Restaurant Name", "Total Reviews", "Reviewer", "Review Text", "Rating"])
    
    if df.empty:
        print("❌ No data scraped! Check the element locators.")
    else:
        df.to_csv("yelp_reviews.csv", index=False)
        print("✅ CSV file saved successfully!")

# Run the function with the given URL
scrape_yelp("https://www.yelp.ca/biz/pai-northern-thai-kitchen-toronto-5?osq=Restaurants")


Restaurant Name: Pai Northern Thai Kitchen
Total Reviews Count: 3788 reviews
❌ No reviews found! Check if the page loads reviews dynamically.
❌ No data scraped! Check the element locators.


In [55]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

def scrape_yelp(url):
    # Set up Selenium WebDriver
    options = Options()
    options.add_argument("--headless")  # Run in headless mode
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    # Open the Yelp page
    driver.get(url)
    time.sleep(5)  # Allow JavaScript to load

    # Extract Restaurant Name
    try:
        restaurant_name = driver.find_element(By.TAG_NAME, "h1").text
        print(f"✅ Restaurant Name: {restaurant_name}")
    except Exception as e:
        print("❌ Failed to get Restaurant Name", e)
        restaurant_name = "Not Found"

    # Extract Total Reviews Count
    try:
        reviews_count = driver.find_element(By.XPATH, "//span[contains(text(),'reviews')]").text
        print(f"✅ Total Reviews Count: {reviews_count}")
    except Exception as e:
        print("❌ Failed to get Total Reviews Count", e)
        reviews_count = "Not Found"

    # Scroll Down to Load More Reviews
    print("🔄 Scrolling to load reviews...")
    body = driver.find_element(By.TAG_NAME, "body")

    for _ in range(8):  # Scroll more times to ensure reviews are loaded
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(3)  # Wait for new reviews to load

    # Wait for reviews to appear
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//li[contains(@class,'y-css-1sqelp2')]"))
        )
    except:
        print("❌ Reviews did not load. Trying alternative methods.")

    # Extract Reviews
    reviews = []
    review_blocks = driver.find_elements(By.XPATH, "//li[contains(@class,'y-css-1sqelp2')]")

    if not review_blocks:
        print("❌ No reviews found! Check the page structure.")
    else:
        print(f"✅ Found {len(review_blocks)} reviews.")

    for block in review_blocks:
        try:
            reviewer = block.find_element(By.XPATH, ".//a[contains(@class,'y-css-1x1e1r2')]").text
        except:
            reviewer = "Anonymous"

        try:
            review_text = block.find_element(By.XPATH, ".//span[contains(@class,'raw__09f24__T4Ezm')]").text
        except:
            review_text = "No Review Text"

        try:
            rating_element = block.find_element(By.XPATH, ".//div[contains(@aria-label,'star rating')]")
            rating = rating_element.get_attribute("aria-label")
        except:
            rating = "No Rating"

        print(f"📌 Review: {review_text[:50]}... | Reviewer: {reviewer} | Rating: {rating}")
        reviews.append([restaurant_name, reviews_count, reviewer, review_text, rating])

    # Close the browser
    driver.quit()

    # Save to CSV
    df = pd.DataFrame(reviews, columns=["Restaurant Name", "Total Reviews", "Reviewer", "Review Text", "Rating"])
    
    if df.empty:
        print("❌ No data scraped! Check the element locators.")
    else:
        df.to_csv("yelp_reviews.csv", index=False)
        print("✅ CSV file saved successfully!")

# Run the function with the given URL
scrape_yelp("https://www.yelp.ca/biz/pai-northern-thai-kitchen-toronto-5?osq=Restaurants")


✅ Restaurant Name: Pai Northern Thai Kitchen
✅ Total Reviews Count: 
🔄 Scrolling to load reviews...
✅ Found 11 reviews.
📌 Review: ... | Reviewer:  | Rating: 5 star rating
📌 Review: ... | Reviewer:  | Rating: 5 star rating
📌 Review: ... | Reviewer:  | Rating: 5 star rating
📌 Review: ... | Reviewer:  | Rating: 4 star rating
📌 Review: ... | Reviewer:  | Rating: 5 star rating
📌 Review: ... | Reviewer:  | Rating: 5 star rating
📌 Review: ... | Reviewer:  | Rating: 5 star rating
📌 Review: ... | Reviewer:  | Rating: 5 star rating
📌 Review: ... | Reviewer:  | Rating: 5 star rating
📌 Review: ... | Reviewer:  | Rating: 5 star rating
📌 Review: No Review Text... | Reviewer: Anonymous | Rating: No Rating
✅ CSV file saved successfully!


In [56]:
df = pd.read_csv("yelp_reviews.csv")
df.head()

,Restaurant Name,Total Reviews,Reviewer,Review Text,Rating
0,Pai Northern Thai Kitchen,NaN,NaN,NaN,5 star rating
1,Pai Northern Thai Kitchen,NaN,NaN,NaN,5 star rating
2,Pai Northern Thai Kitchen,NaN,NaN,NaN,5 star rating
3,Pai Northern Thai Kitchen,NaN,NaN,NaN,4 star rating
4,Pai Northern Thai Kitchen,NaN,NaN,NaN,5 star rating


In [57]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

def scrape_yelp(url):
    # Set up Selenium WebDriver
    options = Options()
    options.add_argument("--headless")  # Run in headless mode
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    # Open the Yelp page
    driver.get(url)
    time.sleep(5)  # Allow JavaScript to load

    # Extract Restaurant Name
    try:
        restaurant_name = driver.find_element(By.TAG_NAME, "h1").text
        print(f"✅ Restaurant Name: {restaurant_name}")
    except Exception as e:
        print("❌ Failed to get Restaurant Name", e)
        restaurant_name = "Not Found"

    # Extract Total Reviews Count
    try:
        reviews_count = driver.find_element(By.XPATH, "//span[contains(text(),'reviews')]").text
        print(f"✅ Total Reviews Count: {reviews_count}")
    except Exception as e:
        print("❌ Failed to get Total Reviews Count", e)
        reviews_count = "Not Found"

    # Scroll Down to Load More Reviews
    print("🔄 Scrolling to load reviews...")
    body = driver.find_element(By.TAG_NAME, "body")

    for _ in range(8):  # Scroll multiple times to load reviews
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(3)  # Wait for new reviews to load

    # Wait for reviews to appear
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//li[contains(@class,'y-css-1sqelp2')]"))
        )
    except:
        print("❌ Reviews did not load. Trying alternative methods.")

    # Extract Reviews
    reviews = []
    review_blocks = driver.find_elements(By.XPATH, "//li[contains(@class,'y-css-1sqelp2')]")

    if not review_blocks:
        print("❌ No reviews found! Check the page structure.")
    else:
        print(f"✅ Found {len(review_blocks)} reviews.")

    for block in review_blocks:
        try:
            # Extracting Reviewer Name
            reviewer = block.find_element(By.XPATH, ".//a[contains(@class,'y-css-1x1e1r2')]").text
        except:
            reviewer = "Anonymous"

        try:
            # Extracting Review Text
            review_text = block.find_element(By.XPATH, ".//span[contains(@class,'raw__09f24__T4Ezm')]").text
        except:
            review_text = "No Review Text"

        try:
            # Extracting Rating (star rating is inside an aria-label attribute)
            rating_element = block.find_element(By.XPATH, ".//div[contains(@aria-label,'star rating')]")
            rating = rating_element.get_attribute("aria-label")
        except:
            rating = "No Rating"

        print(f"📌 Review: {review_text[:50]}... | Reviewer: {reviewer} | Rating: {rating}")
        reviews.append([restaurant_name, reviews_count, reviewer, review_text, rating])

    # Close the browser
    driver.quit()

    # Save to CSV
    df = pd.DataFrame(reviews, columns=["Restaurant Name", "Total Reviews", "Reviewer", "Review Text", "Rating"])
    
    if df.empty:
        print("❌ No data scraped! Check the element locators.")
    else:
        df.to_csv("yelp_reviews.csv", index=False)
        print("✅ CSV file saved successfully!")

# Run the function with the given URL
scrape_yelp("https://www.yelp.ca/biz/pai-northern-thai-kitchen-toronto-5?osq=Restaurants")


✅ Restaurant Name: Pai Northern Thai Kitchen
✅ Total Reviews Count: so businesses can't pay to alter or remove their reviews.
🔄 Scrolling to load reviews...
✅ Found 11 reviews.
📌 Review: It's a really busy restaurant but the staff are ki... | Reviewer: Nhu N. | Rating: 5 star rating
📌 Review: Had a lovely Sunday night dinner at Pai! I was sho... | Reviewer: Celton M. | Rating: 5 star rating
📌 Review: Pai Northern Thai Kitchen offers such a vibrant ex... | Reviewer: Michael C. | Rating: 5 star rating
📌 Review: We arrived on a weekend for dinner around 4:30pm a... | Reviewer: Lisa D. | Rating: 4 star rating
📌 Review: Dude. I think this is the best Thai restaurant Ive... | Reviewer: Angela S. | Rating: 5 star rating
📌 Review: Chicken wings were to die for.. so good, especiall... | Reviewer: Dom G. | Rating: 5 star rating
📌 Review: Wowowowow! What an experience! From start to finis... | Reviewer: Laurel B. | Rating: 5 star rating
📌 Review: Pai Northern Thai Kitchen has got to be one of my 

In [58]:
df_2 = pd.read_csv("yelp_reviews.csv")

In [59]:
df_2.head()

,Restaurant Name,Total Reviews,Reviewer,Review Text,Rating
0,Pai Northern Thai Kitchen,so businesses can't pay to alter or remove the...,Nhu N.,It's a really busy restaurant but the staff ar...,5 star rating
1,Pai Northern Thai Kitchen,so businesses can't pay to alter or remove the...,Celton M.,Had a lovely Sunday night dinner at Pai! I was...,5 star rating
2,Pai Northern Thai Kitchen,so businesses can't pay to alter or remove the...,Michael C.,Pai Northern Thai Kitchen offers such a vibran...,5 star rating
3,Pai Northern Thai Kitchen,so businesses can't pay to alter or remove the...,Lisa D.,We arrived on a weekend for dinner around 4:30...,4 star rating
4,Pai Northern Thai Kitchen,so businesses can't pay to alter or remove the...,Angela S.,Dude. I think this is the best Thai restaurant...,5 star rating


In [60]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

def scrape_yelp_restaurants(url_list, output_file="yelp_reviews.csv"):
    """
    Scrapes reviews from multiple Yelp restaurant pages and saves them to a CSV file.
    """
    # Set up Selenium WebDriver
    options = Options()
    options.add_argument("--headless")  # Run in headless mode
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    all_reviews = []  # Store all reviews

    for url in url_list:
        print(f"🔍 Scraping: {url}")

        # Open the Yelp page
        driver.get(url)
        time.sleep(5)  # Allow JavaScript to load

        # Extract Restaurant Name
        try:
            restaurant_name = driver.find_element(By.TAG_NAME, "h1").text
            print(f"✅ Restaurant Name: {restaurant_name}")
        except:
            restaurant_name = "Not Found"
            print("❌ Failed to get Restaurant Name")

        # Extract Total Reviews Count
        try:
            reviews_count = driver.find_element(By.XPATH, "//span[contains(text(),'reviews')]").text
            print(f"✅ Total Reviews Count: {reviews_count}")
        except:
            reviews_count = "Not Found"
            print("❌ Failed to get Total Reviews Count")

        # Scroll Down to Load More Reviews
        print("🔄 Scrolling to load reviews...")
        body = driver.find_element(By.TAG_NAME, "body")

        for _ in range(8):  # Scroll multiple times to load reviews
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(3)  # Wait for new reviews to load

        # Wait for reviews to appear
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//li[contains(@class,'y-css-1sqelp2')]"))
            )
        except:
            print("❌ Reviews did not load. Trying alternative methods.")

        # Extract Reviews
        review_blocks = driver.find_elements(By.XPATH, "//li[contains(@class,'y-css-1sqelp2')]")

        if not review_blocks:
            print("❌ No reviews found for this restaurant!")
        else:
            print(f"✅ Found {len(review_blocks)} reviews.")

        for block in review_blocks:
            try:
                # Extracting Reviewer Name
                reviewer = block.find_element(By.XPATH, ".//a[contains(@class,'y-css-1x1e1r2')]").text
            except:
                reviewer = "Anonymous"

            try:
                # Extracting Review Text
                review_text = block.find_element(By.XPATH, ".//span[contains(@class,'raw__09f24__T4Ezm')]").text
            except:
                review_text = "No Review Text"

            try:
                # Extracting Rating (star rating is inside an aria-label attribute)
                rating_element = block.find_element(By.XPATH, ".//div[contains(@aria-label,'star rating')]")
                rating = rating_element.get_attribute("aria-label")
            except:
                rating = "No Rating"

            print(f"📌 Review: {review_text[:50]}... | Reviewer: {reviewer} | Rating: {rating}")
            all_reviews.append([restaurant_name, reviews_count, reviewer, review_text, rating])

    # Close the browser
    driver.quit()

    # Save to CSV
    df = pd.DataFrame(all_reviews, columns=["Restaurant Name", "Total Reviews", "Reviewer", "Review Text", "Rating"])

    if df.empty:
        print("❌ No data scraped! Check the element locators.")
    else:
        df.to_csv(output_file, index=False)
        print(f"✅ CSV file '{output_file}' saved successfully with {len(df)} reviews!")

# List of Yelp restaurant URLs to scrape
restaurant_urls = [
    "https://www.yelp.ca/biz/pai-northern-thai-kitchen-toronto-5?osq=Restaurants",
    "https://www.yelp.ca/biz/sukhothai-toronto?osq=Restaurants",
    "https://www.yelp.ca/biz/khao-san-road-toronto?osq=Restaurants"
]

# Run the function with multiple restaurant URLs
scrape_yelp_restaurants(restaurant_urls, "yelp_reviews_multi.csv")


🔍 Scraping: https://www.yelp.ca/biz/pai-northern-thai-kitchen-toronto-5?osq=Restaurants
❌ Failed to get Restaurant Name
❌ Failed to get Total Reviews Count
🔄 Scrolling to load reviews...
❌ Reviews did not load. Trying alternative methods.
❌ No reviews found for this restaurant!
🔍 Scraping: https://www.yelp.ca/biz/sukhothai-toronto?osq=Restaurants
✅ Restaurant Name: Sukhothai
✅ Total Reviews Count: so businesses can't pay to alter or remove their reviews.
🔄 Scrolling to load reviews...
✅ Found 11 reviews.
📌 Review: Very packed at rush hour around 6.30pm. They sat u... | Reviewer: Raya S. | Rating: 3 star rating
📌 Review: I'm updating my review to a 1 star because the ser... | Reviewer: Kerr N. | Rating: 1 star rating
📌 Review: Not sure why I haven't written an review for this ... | Reviewer: Carmen C. | Rating: 4 star rating
📌 Review: I've been here a few times and find the food excel... | Reviewer: Lester C. | Rating: 4 star rating
📌 Review: A great little Thai eatery on Parliament St 

In [61]:
df_3 = pd.read_csv("yelp_reviews_multi.csv")

In [63]:
df_3.head(40)

,Restaurant Name,Total Reviews,Reviewer,Review Text,Rating
0,Sukhothai,so businesses can't pay to alter or remove the...,Raya S.,Very packed at rush hour around 6.30pm. They s...,3 star rating
1,Sukhothai,so businesses can't pay to alter or remove the...,Kerr N.,I'm updating my review to a 1 star because the...,1 star rating
2,Sukhothai,so businesses can't pay to alter or remove the...,Carmen C.,Not sure why I haven't written an review for t...,4 star rating
3,Sukhothai,so businesses can't pay to alter or remove the...,Lester C.,I've been here a few times and find the food e...,4 star rating
4,Sukhothai,so businesses can't pay to alter or remove the...,Jay L.,A great little Thai eatery on Parliament St an...,4 star rating
5,Sukhothai,so businesses can't pay to alter or remove the...,Jessica E.,A must on anybody's Toronto best eats bucket l...,5 star rating
6,Sukhothai,so businesses can't pay to alter or remove the...,Peter T.,I love this little place. I've been coming for...,4 star rating
7,Sukhothai,so businesses can't pay to alter or remove the...,Phillip S.,There are a few location in the city where Suk...,4 star rating
8,Sukhothai,so businesses can't pay to alter or remove the...,Karen M.,I've been to the Wellington location of Sukhot...,4 star rating
9,Sukhothai,so businesses can't pay to alter or remove the...,Jennifer K.,"If not for Yelp, I would have never ever go to...",4 star rating
